https://www.kaggle.com/c/word2vec-nlp-tutorial

labeled data를 전처리하고 df에 담자

In [1]:
import pandas as pd

train = pd.read_csv('./data/labeledTrainData.tsv',
    header=0,
    delimiter='\t',
    quoting=3)

# quoting : int or csv.QUOTE_* instance, default 0
#     Control field quoting behavior per ``csv.QUOTE_*`` constants. Use one of
#     QUOTE_MINIMAL (0), QUOTE_ALL (1), QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).
# 3으로 주었으니까 quoting을 무시하는 옵션임

train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
test = pd.read_csv('./data/testData.tsv',
    header=0,
    delimiter='\t',
    quoting=3)

test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [12]:
train.columns # ['id', 'sentiment', 'review']
train.review[0][:700]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely lik'

데이터 정제  

(영어 기준)  
1. html 태그 제거  
2. 정규 표현식으로 특수 문자 제거  
3. stopword 제거  
4. 어간 추출 (stemming)  

In [14]:
# 태그 제거
from bs4 import BeautifulSoup

# html.parser 파서 사용하여 정제
extractHtmlTags = BeautifulSoup(train.review[0], 'html.parser').get_text()
extractHtmlTags

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [16]:
# 정규 표현식으로 특수 문자 제거
import re

letters_only = re.sub("[^a-zA-Z0-9]", " ", extractHtmlTags)
letters_only[:500]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle mess'

In [18]:
# 모두 소문자로 변환
lower_case = letters_only.lower()
lower_case[:700]

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyw'

In [25]:
# 단어 토큰으로 쪼개기
from nltk import word_tokenize
tokenized_word= word_tokenize(lower_case)
len(tokenized_word)

438

In [38]:
# 불용어 제거
from nltk.corpus import stopwords

meaingful_words = [w for w in tokenized_word if not w in stopwords.words("english")]
len(meaingful_words) # 불용어 제거하니까 220개밖에 안 남음
meaingful_words[:2]

['stuff', 'going']

In [39]:
# stemming (어간 추출)
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
words = [stemmer.stem(w) for w in meaingful_words]
words[:2] # stemming 처리가 된 words -> going -> go, documentary -> documentari 뭐 이런식으로 됨

['stuff', 'go']

In [42]:
# Lemmatize
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

words = [wordnet_lemmatizer.lemmatize(w) for w in words]
words[:2]

['stuff', 'go']

In [49]:
# 위 작업을 함수로 묶어 봤음
def raw_review_to_words(raw_review):
    extractHtmlTags = BeautifulSoup(raw_review, 'html.parser').get_text()
    letters_only = re.sub("[^a-zA-Z0-9]", " ", extractHtmlTags)
    lower_case = letters_only.lower()
    tokenized_word= word_tokenize(lower_case)
    meaingful_words = [w for w in tokenized_word if not w in stopwords.words("english")]
    stems = [stemmer.stem(w) for w in meaingful_words]
    words = [wordnet_lemmatizer.lemmatize(w) for w in stems]
    return words

In [64]:
# 위와 같은 작업을 모든 train.reviews에 해줘야 함
print(train.review.size) # 25000개의 리뷰에 대해서 처리
print(test.review.size) # 25000개의 리뷰에 대해서 처리

25000
25000


In [79]:
# https://gist.github.com/yong27/7869662
# ipython 사용하는 주피터 노트북에서 멀티프로세싱이 안되는 이슈가 있음. workaround가 있긴 한데 아래 글을 참고해보시고, 
# https://github.com/ipython/ipython/issues/10894
# 나는 일단 패스할거임... ㅜ
from multiprocessing import Pool
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = Pool(processes=workers)
    print("workers number: ", workers)
    print("pool", pool)
    
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

# 아래 꼴로 진행하려고 했는데 ipython 환경에서 멀티 프로세싱이 안 먹는데;;
# golang 쓰고 싶어 진다 으아ㅏ아앙
# apply_by_multiprocessing(test.review, raw_review_to_words, workers=4)

In [82]:
train['review_clean'] = train.review.apply(raw_review_to_words)

In [ ]:
train.head(2) # 어쨌든 아래처럼 생겼다

In [85]:
test['review_clean'] = test.review.apply(raw_review_to_words)